In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "E:/my_courses/data mining/project/data_genre/"
path = "C:/Users/User/Downloads/test2/"

In [3]:
book = pd.read_csv(path + 'book_no_null_language_merging.csv')

# 2️⃣ Data Preparation Phase : Section 3
<hr style="border:2px solid gray"> </hr>

## 📑 Using genres field

### ✅ Joining books with genres

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
در این قسمت قصد داریم که ژانرهای مختلف هرکتاب را به دیتاست کتابی که داشتیم اضافه کنیم .
برای انجام این امر، از دیتاست دیگری که برگرفته از سایت goodreads میباشد استفاده میکنیم چراکه دیتاست اصلی حاوی ژانر نمی باشد.

In [3]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9924 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  10000 non-null  float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              10000 non-null  object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [4]:
genres_df = pd.read_json('goodreads_book_genres_initial.json', lines=True)

In [5]:
genres_df.columns = ['goodreads_book_id','genres']

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
در دیتاست زیر هر کتاب دارای لیستی از ژانرهایی است که بیشتر به آن رای داده اند مثلا اولین سطر نشان میدهد که این کتاب تاریخی میباشد.

In [6]:
genres_df.head()

,goodreads_book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
دیتاست کتاب را با دیتاست ژانر ذکر شده جوین میکنیم.

In [10]:
book_genres_df = pd.merge(book, genres_df, how='left', on=['goodreads_book_id'])

In [11]:
book_genres_df.shape

(10000, 24)

In [12]:
book_genres_df.drop(['goodreads_book_id', 'best_book_id','work_id','books_count','isbn',
                     'isbn13','original_title','authors','image_url','small_image_url','title'], axis=1, inplace=True)

In [13]:
book_genres_df.shape

(10000, 13)

In [14]:
book_genres_df.to_csv(path + 'genre_book.csv')

### ✅ Separating genres

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
در این قسمت ژانرها را جدا می کنیم تا در کل بفهمیم کتاب ها دارای چه نوع ژانرهایی
    هستند. یک کتاب می تواند دارای چنیدن ژانر باشد.

In [14]:
book_genres_df = pd.read_csv(path + 'genre_book.csv')

In [15]:
book_genres_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [16]:
book_genres_df.loc[0,'genres']

"{'young-adult': 30173, 'fiction': 26304, 'fantasy, paranormal': 13609, 'romance': 3390, 'mystery, thriller, crime': 803}"

In [17]:
import json

genres_list = set()
for i in range(len(book_genres_df)):
    if str(book_genres_df.loc[i,'genres']) != 'nan':
        genres = json.loads(book_genres_df.loc[i,'genres'].replace("'","\"")).keys()
        for j in genres:
            genres_list.add(j)

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
    همانطور که در زیر دیده میشود ژانرها به صورت زیر میباشد . بعضی ها به صورت تکی مانند ژانر کودک و بعضی از آنها به صورت گروهی در نظر گرفته شده اند مانند بایوگرافی ، تاریخی و تاریخی تخیلی

In [18]:
genres_list

{'children',
 'comics, graphic',
 'fantasy, paranormal',
 'fiction',
 'history, historical fiction, biography',
 'mystery, thriller, crime',
 'non-fiction',
 'poetry',
 'romance',
 'young-adult'}

In [19]:
len(genres_list)

10

### ✅ Creating one-hot genre

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
    حال در این قسمت یک one-hot از ژانرهایی که در قسمت قبل بدست آوردیم میسازیم.
    به این دیتاست جدید آیدی کتاب ها را نیز اضافه میکنیم.

In [20]:
onehot = pd.DataFrame(columns=genres_list)
onehot['book_id'] = book_genres_df['book_id']
onehot.head()

,"fantasy, paranormal",non-fiction,"mystery, thriller, crime",fiction,young-adult,children,poetry,"comics, graphic",romance,"history, historical fiction, biography",book_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [21]:
for i in range(len(book_genres_df)):
    if pd.isnull(book_genres_df.loc[i,'genres']) != True:
        genres = json.loads(book_genres_df.loc[i,'genres'].replace("'","\"")).keys()
        for j in genres:
            onehot.loc[i,j] = 1

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
    در نهایت دیتاست به صورت زیر بدست می آید به طوریکه به ازای هر کتاب ژانرهایی که داشته است مقدار یک دارند و بقیه صفر هستند.

In [22]:
onehot = onehot.fillna(0)
onehot.head()

,"fantasy, paranormal",non-fiction,"mystery, thriller, crime",fiction,young-adult,children,poetry,"comics, graphic",romance,"history, historical fiction, biography",book_id
0,1,0,1,1,1,0,0,0,1,0,1
1,1,0,1,1,1,1,0,0,0,0,2
2,1,0,0,1,1,0,0,0,1,0,3
3,0,0,1,1,1,0,0,0,0,1,4
4,0,0,0,1,1,0,0,0,1,1,5


### ✅ Imputation of null genre manually

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
    در بین ژانرهایی که در دیتاست ژانر در قسمت های قبل دیده شد ، چند کتاب ژانری برایشان ثبت نشده بود پس داده های گم شده بودند .
    در این قسمت این داده های گم شده را با استفاده از ویژگی های کتاب مورد نظر ، به صورت دستی جستجو و اپدیت میکنیم .

In [25]:
book_genres_df[book_genres_df['genres'].isna()]

,book_id,original_publication_year,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,genres
7195,7196,2016.0,en,4.13,17375,22369,2309,299,726,3616,8891,8837,NaN
8700,8701,1991.0,es-ar,4.21,9115,9116,4,195,366,1301,2761,4493,NaN
9607,9608,1997.0,en,4.14,11106,11106,3,125,277,2425,3384,4895,NaN
9815,9816,2016.0,en,4.23,10891,13808,1655,148,438,1925,4879,6418,NaN


In [26]:
onehot.loc[7195]

fantasy, paranormal                          0
non-fiction                                  0
mystery, thriller, crime                     0
fiction                                      0
young-adult                                  0
children                                     0
poetry                                       0
comics, graphic                              0
romance                                      0
history, historical fiction, biography       0
book_id                                   7196
Name: 7195, dtype: int64

In [27]:
onehot.loc[7195,['romance','fiction']] = 1
onehot.loc[8700,['comics, graphic','fiction']] = 1
onehot.loc[9607,['children','fiction','young-adult']] = 1
onehot.loc[9815,['romance','fiction']] = 1

### ✅ Merge onehot with book_genres_df

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
    در نهایت دیتاست one-hot .ژانری که بدست آمد را با دیتاست کتاب جوین زده و فیلد genres قبلی از دیتاست حذف می گردد. 

In [28]:
one_hot_genre = pd.merge(book_genres_df, onehot, how='inner', on=['book_id'])

In [29]:
one_hot_genre.head()

,book_id,original_publication_year,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,...,"fantasy, paranormal",non-fiction,"mystery, thriller, crime",fiction,young-adult,children,poetry,"comics, graphic",romance,"history, historical fiction, biography"
0,1,2008.0,en,4.34,4780653,4942365,155254,66715,127936,560092,...,1,0,1,1,1,0,0,0,1,0
1,2,1997.0,en,4.44,4602479,4800065,75867,75504,101676,455024,...,1,0,1,1,1,1,0,0,0,0
2,3,2005.0,en,3.57,3866839,3916824,95009,456191,436802,793319,...,1,0,0,1,1,0,0,0,1,0
3,4,1960.0,en,4.25,3198671,3340896,72586,60427,117415,446835,...,0,0,1,1,1,0,0,0,0,1
4,5,1925.0,en,3.89,2683664,2773745,51992,86236,197621,606158,...,0,0,0,1,1,0,0,0,1,1


In [30]:
one_hot_genre.drop(['genres'], inplace=True, axis=1)

### ✅ Get dummy language_code

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
اگر تعداد زبان های unique را بدست آوریم درون این دیتاست کتاب هایی به 25 زبان گوناگون وجود دارند اما اگر به توزیع این زبان ها نگاهی بیندازیم مشخص است که 14 مورد حداکثر 3 کتاب درون دیتاست دارند و تعداد زیادی نیز تنها یک کتاب را شامل می شوند به این ترتیب میتوانیم یک خوشه بندی بر روی زبان هایی که کتاب کمتری دارند انجام دهیم.

In [37]:
len(one_hot_genre['language_code'].unique())

25

In [38]:
one_hot_genre['language_code'].value_counts()

en       9799
ara        64
fre        29
ind        21
spa        20
ger        15
jpn         9
per         7
pol         6
por         6
dan         3
nor         3
swe         2
ita         2
fil         2
hin         2
tur         2
vie         1
rus         1
chi         1
rum         1
mul         1
nl          1
fin         1
es-ar       1
Name: language_code, dtype: int64

In [39]:
for i in one_hot_genre[~book['language_code'].isna()]['language_code'].unique():
    if (one_hot_genre['language_code'].value_counts()[i] < 6):
        one_hot_genre['language_code'].replace({i:'other'}, inplace=True)

In [40]:
len(one_hot_genre['language_code'].unique())

11

In [41]:
one_hot_genre['language_code'].value_counts()

en       9799
ara        64
fre        29
other      24
ind        21
spa        20
ger        15
jpn         9
per         7
pol         6
por         6
Name: language_code, dtype: int64

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
پس از دسته بندی زبان ها و کاهش تعداد ستون های مربوطه از dummy استفاده کرده و ستون کتگوریکال language-code را به ستون های flag مانند تبدیل می کنیم.

In [42]:
final_book =  pd.get_dummies(one_hot_genre, columns=['language_code'] , prefix=[''],prefix_sep=[''])
final_book.head()

,book_id,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,...,en,fre,ger,ind,jpn,other,per,pol,por,spa
0,1,2008.0,4.34,4780653,4942365,155254,66715,127936,560092,1481305,...,1,0,0,0,0,0,0,0,0,0
1,2,1997.0,4.44,4602479,4800065,75867,75504,101676,455024,1156318,...,1,0,0,0,0,0,0,0,0,0
2,3,2005.0,3.57,3866839,3916824,95009,456191,436802,793319,875073,...,1,0,0,0,0,0,0,0,0,0
3,4,1960.0,4.25,3198671,3340896,72586,60427,117415,446835,1001952,...,1,0,0,0,0,0,0,0,0,0
4,5,1925.0,3.89,2683664,2773745,51992,86236,197621,606158,936012,...,1,0,0,0,0,0,0,0,0,0


In [43]:
print(list(final_book.columns))

['book_id', 'original_publication_year', 'average_rating', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'fantasy, paranormal', 'non-fiction', 'mystery, thriller, crime', 'fiction', 'young-adult', 'children', 'poetry', 'comics, graphic', 'romance', 'history, historical fiction, biography', 'ara', 'en', 'fre', 'ger', 'ind', 'jpn', 'other', 'per', 'pol', 'por', 'spa']


In [44]:
final_book.to_csv(path + 'final_book.csv')